In [163]:
# Import Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [164]:
# Load Dataset
df = pd.read_csv('all_months_clean.csv', sep=';')

In [165]:
# Cek data teratas
df.head()

,order_id,total_qty,total_weight_gr,total_returned_qty,Total Diskon,product_categories,num_product_categories,Status Pesanan,Alasan Pembatalan,Opsi Pengiriman,Metode Pembayaran,Kota/Kabupaten,Provinsi,Ongkos Kirim Dibayar oleh Pembeli,Estimasi Potongan Biaya Pengiriman,Total Pembayaran,Perkiraan Ongkos Kirim,Waktu Pesanan Dibuat,source_file
0,ORD_0000001,2,2000,0,0,Celengan,1,Selesai,NaN,Reguler (Cashless)-SPX Standard,Saldo ShopeePay,KOTA SERANG,BANTEN,0,10000,38300,10000,2024-04-01 00:15,AprilSales2024.xlsx
1,ORD_0000002,1,500,0,0,Celengan,1,Selesai,NaN,Hemat Kargo-SPX Hemat,COD (Bayar di Tempat),KOTA SEMARANG,JAWA TENGAH,0,14500,18576,14500,2024-04-01 01:47,AprilSales2024.xlsx
2,ORD_0000003,1,500,0,0,Celengan,1,Selesai,NaN,Hemat Kargo-SPX Hemat,SeaBank Bayar Instan,KAB. BOGOR,JAWA BARAT,0,8000,7069,8000,2024-04-01 04:25,AprilSales2024.xlsx
3,ORD_0000004,2,400,0,0,Mangkok Sambal / Saus,1,Selesai,NaN,Hemat Kargo-SPX Hemat,COD (Bayar di Tempat),KOTA JAMBI,JAMBI,0,20000,32200,20000,2024-04-01 04:41,AprilSales2024.xlsx
4,ORD_0000005,3,3600,0,0,"Keranjang, Other, Tempat Nasi",3,Batal,Dibatalkan oleh Pembeli. Alasan: Ubah Pesanan ...,Hemat Kargo-SPX Hemat,COD (Bayar di Tempat),KOTA TANGERANG,BANTEN,0,0,0,8000,2024-04-01 06:12,AprilSales2024.xlsx


In [166]:
# Memeriksa kolom dan tipe data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20848 entries, 0 to 20847
Data columns (total 19 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   order_id                            20848 non-null  object
 1   total_qty                           20848 non-null  int64 
 2   total_weight_gr                     20848 non-null  int64 
 3   total_returned_qty                  20848 non-null  int64 
 4   Total Diskon                        20848 non-null  int64 
 5   product_categories                  20848 non-null  object
 6   num_product_categories              20848 non-null  int64 
 7   Status Pesanan                      20848 non-null  object
 8   Alasan Pembatalan                   2830 non-null   object
 9   Opsi Pengiriman                     20848 non-null  object
 10  Metode Pembayaran                   20848 non-null  object
 11  Kota/Kabupaten                      20848 non-null  ob

In [167]:
# Memperbaiki tipe data waktu dan membuat turunan waktu meliputi Tanggal, Bulan, Hari, Jam menit
# Hasil Perbaikan kolom waktu pesanan disimpan pada kolom waktu kemudian diturunkan menjadi beberapa hierarki waktu
df['Waktu'] = df['Waktu Pesanan Dibuat'].ffill()
df['Waktu'] = pd.to_datetime(df['Waktu'])
df['hari']  = df['Waktu'].dt.day
df['bulan'] = df['Waktu'].dt.month
df['tahun'] = df['Waktu'].dt.year
df['jam']   = df['Waktu'].dt.hour

In [168]:
# Memeriksa Hasil perbaikan
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20848 entries, 0 to 20847
Data columns (total 24 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   order_id                            20848 non-null  object        
 1   total_qty                           20848 non-null  int64         
 2   total_weight_gr                     20848 non-null  int64         
 3   total_returned_qty                  20848 non-null  int64         
 4   Total Diskon                        20848 non-null  int64         
 5   product_categories                  20848 non-null  object        
 6   num_product_categories              20848 non-null  int64         
 7   Status Pesanan                      20848 non-null  object        
 8   Alasan Pembatalan                   2830 non-null   object        
 9   Opsi Pengiriman                     20848 non-null  object        
 10  Metode Pembayaran     

In [169]:
# Melakukan groupby alasan pembatalan dan opsi pengiriman berdasarkan banyaknya pelanggan
# ini dilakukan untuk mengetahui opsi pengiriman apa saja yang memiliki alasan pembatalan paling banyak terkait keterlambatan pengiriman
df.groupby(['Alasan Pembatalan', 'Opsi Pengiriman'])['order_id'].nunique()

Alasan Pembatalan                                                                                                                                   Opsi Pengiriman                                      
Dibatalkan oleh Pembeli. Alasan: Lainnya                                                                                                            GrabExpress Instant                                        1
                                                                                                                                                    Hemat Kargo                                               26
                                                                                                                                                    Hemat Kargo-SPX Hemat                                      1
                                                                                                                                                    Instant (Versi Lama)   

In [170]:
# Menggroupby perkiraan ongkos kirim berdasarkan provinsi, ini dilakukan untuk mengetahui provinsi mana yang memiliki ongkir tertinggi
df.groupby('Provinsi')['Perkiraan Ongkos Kirim'].mean().astype(int).sort_values(ascending=False)

Provinsi
PAPUA BARAT                       123000
PAPUA                             115000
MALUKU UTARA                       94644
NUSA TENGGARA TIMUR (NTT)          92533
SULAWESI TENGGARA                  91815
MALUKU                             77465
SULAWESI UTARA                     71907
SULAWESI BARAT                     63506
SULAWESI TENGAH                    56210
NUSA TENGGARA BARAT (NTB)          52939
KALIMANTAN TIMUR                   52122
KALIMANTAN UTARA                   50491
KEPULAUAN RIAU                     47421
KALIMANTAN TENGAH                  46357
SULAWESI SELATAN                   46104
GORONTALO                          42925
KALIMANTAN SELATAN                 41071
KALIMANTAN BARAT                   39475
SUMATERA UTARA                     37843
BALI                               36346
NANGGROE ACEH DARUSSALAM (NAD)     34482
BANGKA BELITUNG                    32975
RIAU                               32475
JAMBI                              31323
SUMATER

In [171]:
# Groupby estimasi potongan berdasarkan provinsi untuk mengetahui apakah provinsi yang masuk kategori dengan ongkos kirim tertinggi sudah mendapatkan potongan semestinya
df.groupby('Provinsi')['Estimasi Potongan Biaya Pengiriman'].mean().astype(int).sort_values(ascending=False)

Provinsi
PAPUA BARAT                       40000
PAPUA                             40000
SULAWESI TENGGARA                 34156
MALUKU                            30250
MALUKU UTARA                      24444
KALIMANTAN UTARA                  23425
SULAWESI SELATAN                  22502
BALI                              21667
KEPULAUAN RIAU                    21459
NUSA TENGGARA BARAT (NTB)         21066
KALIMANTAN TENGAH                 20987
SUMATERA UTARA                    20814
SULAWESI UTARA                    20158
BANGKA BELITUNG                   19833
JAMBI                             19755
KALIMANTAN SELATAN                19311
NUSA TENGGARA TIMUR (NTT)         19166
KALIMANTAN BARAT                  19133
RIAU                              19118
NANGGROE ACEH DARUSSALAM (NAD)    18897
KALIMANTAN TIMUR                  18771
SULAWESI BARAT                    18750
GORONTALO                         18634
SUMATERA BARAT                    17509
SUMATERA SELATAN               

In [172]:
# menggroupby ongkos kirim sebelum dikenakan potongan ongkir
df.groupby('Provinsi')['Ongkos Kirim Dibayar oleh Pembeli'].mean().astype(int).sort_values(ascending=False)

Provinsi
PAPUA BARAT                       83000
PAPUA                             75000
MALUKU UTARA                      54144
SULAWESI TENGGARA                 34050
SULAWESI BARAT                    32212
MALUKU                            27390
NUSA TENGGARA TIMUR (NTT)         24941
SULAWESI UTARA                    22899
SULAWESI TENGAH                   20544
KALIMANTAN UTARA                  18491
KALIMANTAN TIMUR                  15542
NUSA TENGGARA BARAT (NTB)         14701
GORONTALO                         12207
KALIMANTAN SELATAN                11875
SULAWESI SELATAN                  11686
KALIMANTAN TENGAH                 11433
KALIMANTAN BARAT                  11316
KEPULAUAN RIAU                     8779
BALI                               8622
BANGKA BELITUNG                    8478
SUMATERA UTARA                     8256
SUMATERA SELATAN                   6919
NANGGROE ACEH DARUSSALAM (NAD)     6840
BENGKULU                           6594
RIAU                           

In [173]:
# Melihat daya beli setiap provinsi berdasarkan total pembayaran setiap provinsi
df.groupby('Provinsi')['Total Pembayaran'].sum().astype(int).sort_values(ascending=False)

Provinsi
JAWA BARAT                        276992073
BANTEN                            177861752
DKI JAKARTA                       135164841
JAWA TENGAH                       100114044
JAWA TIMUR                         95478924
SUMATERA SELATAN                   40909428
LAMPUNG                            22028652
BALI                               21951090
SUMATERA UTARA                     19783218
SULAWESI SELATAN                   19680129
JAMBI                              15651007
RIAU                               15011507
NUSA TENGGARA BARAT (NTB)          13047395
KALIMANTAN BARAT                   12360930
SUMATERA BARAT                     11217328
KALIMANTAN SELATAN                 10075885
KALIMANTAN TIMUR                    9446377
DI YOGYAKARTA                       9422267
KALIMANTAN TENGAH                   6949690
BENGKULU                            6576117
BANGKA BELITUNG                     6328579
NANGGROE ACEH DARUSSALAM (NAD)      5936568
SULAWESI TENGGARA      

In [174]:
# Melihat jumlah pembatalan pesanan berdasrkan provinsi
df.groupby('Provinsi')['Status Pesanan'].apply(lambda x: (x=='Batal').sum()).sort_values(ascending=False)

Provinsi
JAWA BARAT                        754
BANTEN                            433
DKI JAKARTA                       361
JAWA TIMUR                        251
JAWA TENGAH                       188
SUMATERA SELATAN                  101
LAMPUNG                            69
RIAU                               61
SUMATERA UTARA                     60
SULAWESI SELATAN                   58
JAMBI                              55
SUMATERA BARAT                     52
KALIMANTAN BARAT                   40
KALIMANTAN TIMUR                   37
BALI                               35
KALIMANTAN TENGAH                  33
NANGGROE ACEH DARUSSALAM (NAD)     32
KALIMANTAN SELATAN                 31
NUSA TENGGARA BARAT (NTB)          29
DI YOGYAKARTA                      28
BENGKULU                           23
KEPULAUAN RIAU                     17
SULAWESI TENGAH                    17
BANGKA BELITUNG                    16
SULAWESI UTARA                     13
GORONTALO                          11
SUL

In [175]:
# Mengetahui lebih detail alasan pembatalan
df.groupby(['Provinsi', 'Alasan Pembatalan'])['order_id'].count()

Provinsi                        Alasan Pembatalan                                                                                                                                 
BALI                            Dibatalkan oleh Pembeli. Alasan: Lainnya                                                                                                                1
                                Dibatalkan oleh Pembeli. Alasan: Lainnya/ berubah pikiran                                                                                               8
                                Dibatalkan oleh Pembeli. Alasan: Need to change delivery address                                                                                        2
                                Dibatalkan oleh Pembeli. Alasan: Perlu mengubah alamat pengiriman                                                                                       1
                                Dibatalkan oleh Pembeli. Alasan: Proses pemba

In [176]:
# Melihat rata rata potongan harga berdasarkan opsi pengiriman
df.groupby('Opsi Pengiriman')['Estimasi Potongan Biaya Pengiriman'].mean().sort_values(ascending=False)

Opsi Pengiriman
JNE Trucking (JTR)                                       74750.000000
J&T Cargo                                                28881.491803
Hemat Kargo-JNE Trucking (JTR)                           26687.500000
Hemat Kargo-J&T Cargo                                    26055.555556
Instant (Versi Lama)-GoSend Instant (Versi Lama)         25318.181818
Kargo-J&T Cargo                                          21692.040000
SPX Instant (Versi Lama)                                 20994.500000
Instant (Versi Lama)-SPX Instant (Versi Lama)            20172.619048
Gosend Instant                                           20000.000000
Instant (Versi Lama)-GrabExpress Instant (Versi Lama)    19818.181818
Kargo-JNE Trucking (JTR)                                 18613.715789
SPX Instant                                              17000.000000
Instant-SPX Instant                                      16475.000000
GoSend Instant (Versi Lama)                              16300.000000
Hema

In [177]:
df.groupby(['Provinsi', 'Opsi Pengiriman'])['order_id'].count().sort_values(ascending=False)

Provinsi                        Opsi Pengiriman                                      
JAWA BARAT                      Hemat Kargo-SPX Hemat                                    4370
BANTEN                          Hemat Kargo-SPX Hemat                                    1799
JAWA BARAT                      Reguler (Cashless)-SPX Standard                          1622
DKI JAKARTA                     Hemat Kargo-SPX Hemat                                    1406
JAWA TIMUR                      Hemat Kargo-SPX Hemat                                    1050
JAWA TENGAH                     Hemat Kargo-SPX Hemat                                     970
BANTEN                          Reguler (Cashless)-SPX Standard                           945
DKI JAKARTA                     Reguler (Cashless)-SPX Standard                           890
SUMATERA SELATAN                Hemat Kargo-SPX Hemat                                     401
JAWA TIMUR                      Reguler (Cashless)-SPX Standard     

In [178]:
df[df['Provinsi'] == 'PAPUA BARAT'].groupby('Opsi Pengiriman')['order_id'].count().sort_values(ascending=False)

Opsi Pengiriman
Hemat Kargo-SPX Hemat    1
Name: order_id, dtype: int64

In [179]:
df.groupby('Provinsi')['Total Diskon'].mean().sort_values(ascending=False)

Provinsi
SUMATERA SELATAN                  7177.429467
SULAWESI TENGGARA                 1218.750000
SULAWESI SELATAN                   744.197674
JAWA TIMUR                         497.187617
BANTEN                             382.586121
LAMPUNG                            316.878601
DKI JAKARTA                        310.420145
BENGKULU                           204.255319
KEPULAUAN RIAU                      81.081081
JAWA BARAT                          48.306273
BALI                                42.424242
NANGGROE ACEH DARUSSALAM (NAD)      42.091603
DI YOGYAKARTA                       39.686508
RIAU                                24.413690
SUMATERA BARAT                      23.297491
KALIMANTAN BARAT                    22.325581
JAWA TENGAH                          3.022727
SULAWESI TENGAH                      0.053571
SUMATERA UTARA                       0.019417
KALIMANTAN TENGAH                    0.014925
KALIMANTAN SELATAN                   0.006579
BANGKA BELITUNG          

In [180]:
df.groupby('Provinsi')['total_weight_gr'].mean().sort_values(ascending=False)

Provinsi
KEPULAUAN RIAU                    3214.054054
NUSA TENGGARA BARAT (NTB)         3027.666667
BALI                              2649.823232
SUMATERA SELATAN                  2585.000000
DKI JAKARTA                       2564.008279
BANTEN                            2496.435720
PAPUA                             2400.000000
MALUKU UTARA                      2377.777778
NUSA TENGGARA TIMUR (NTT)         2373.750000
MALUKU                            2170.000000
SULAWESI BARAT                    2151.250000
RIAU                              2139.761905
SULAWESI UTARA                    2085.121951
JAWA TIMUR                        2078.533458
KALIMANTAN TIMUR                  2076.616541
SULAWESI SELATAN                  1984.496124
JAMBI                             1955.029240
GORONTALO                         1931.730769
SUMATERA UTARA                    1905.906149
KALIMANTAN UTARA                  1851.666667
JAWA TENGAH                       1817.870989
PAPUA BARAT              

In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20848 entries, 0 to 20847
Data columns (total 24 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   order_id                            20848 non-null  object        
 1   total_qty                           20848 non-null  int64         
 2   total_weight_gr                     20848 non-null  int64         
 3   total_returned_qty                  20848 non-null  int64         
 4   Total Diskon                        20848 non-null  int64         
 5   product_categories                  20848 non-null  object        
 6   num_product_categories              20848 non-null  int64         
 7   Status Pesanan                      20848 non-null  object        
 8   Alasan Pembatalan                   2830 non-null   object        
 9   Opsi Pengiriman                     20848 non-null  object        
 10  Metode Pembayaran     

In [182]:
df.drop(columns=['source_file'], inplace=True)

In [183]:
df.to_csv('ranalysis.csv')

In [184]:
df.describe()

,total_qty,total_weight_gr,total_returned_qty,Total Diskon,num_product_categories,Ongkos Kirim Dibayar oleh Pembeli,Estimasi Potongan Biaya Pengiriman,Total Pembayaran,Perkiraan Ongkos Kirim,Waktu,hari,bulan,tahun,jam
count,20848.000000,20848.000000,20848.000000,20848.000000,20848.000000,20848.000000,20848.000000,2.084800e+04,20848.000000,20848,20848.000000,20848.000000,20848.000000,20848.000000
mean,2.560821,2004.529691,0.017748,405.199348,1.112097,4189.990695,10722.539284,5.068360e+04,18425.626343,2024-12-03 17:29:27.087490304,16.557703,7.109987,2024.371450,14.287222
min,1.000000,10.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000e+00,1.000000,2023-12-01 00:33:00,1.000000,1.000000,2023.000000,0.000000
25%,1.000000,300.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.348000e+04,8000.000000,2024-07-11 16:31:30,8.000000,4.000000,2024.000000,10.000000
50%,1.000000,500.000000,0.000000,0.000000,1.000000,0.000000,9500.000000,2.180000e+04,11000.000000,2024-11-01 13:48:30,16.000000,8.000000,2024.000000,15.000000
75%,2.000000,1600.000000,0.000000,0.000000,1.000000,3000.000000,15000.000000,4.160000e+04,20000.000000,2025-05-24 10:54:30,25.000000,10.000000,2025.000000,19.000000
max,256.000000,375000.000000,70.000000,700000.000000,11.000000,584000.000000,312000.000000,3.403591e+06,959200.000000,2025-11-30 23:17:00,31.000000,12.000000,2025.000000,23.000000
std,7.796763,7106.357515,0.548656,9784.017705,0.485361,13578.489678,12672.471066,1.440193e+05,23791.508665,NaN,9.614597,3.362530,0.626538,5.722137


In [185]:
# lakukan kombinasi analisis korelasi dan persebaran untuk menilai apakah insentif berupa diskon dan subsidi ongkir telah terdistribusi secara proporsional terhadap beban ongkir dan nilai transaksi di setiap wilayah
korelasi_distribusi = df.groupby('Provinsi').agg({
    'Ongkos Kirim Dibayar oleh Pembeli': 'mean',
    'Estimasi Potongan Biaya Pengiriman': 'mean',
    'Perkiraan Ongkos Kirim': 'mean',
    'Total Pembayaran': 'sum',
    'Total Diskon': 'mean',
    'order_id': 'count'
}).reset_index()

korelasi_distribusi

,Provinsi,Ongkos Kirim Dibayar oleh Pembeli,Estimasi Potongan Biaya Pengiriman,Perkiraan Ongkos Kirim,Total Pembayaran,Total Diskon,order_id
0,BALI,8622.636364,21667.904040,36346.292929,21951090,42.424242,198
1,BANGKA BELITUNG,8478.746032,19833.023810,32975.571429,6328579,0.000000,126
2,BANTEN,3231.534974,7944.435997,13131.855129,177861752,382.586121,3617
3,BENGKULU,6594.602837,16256.936170,30376.319149,6576117,204.255319,141
4,DI YOGYAKARTA,2671.031746,10994.841270,17499.603175,9422267,39.686508,252
5,DKI JAKARTA,3275.858227,7470.983098,12967.947223,135164841,310.420145,2899
6,GORONTALO,12207.692308,18634.615385,42925.000000,2229815,0.000000,52
7,JAMBI,4904.909357,19755.847953,31323.096491,15651007,0.000000,342
8,JAWA BARAT,2393.450630,7911.702131,11781.405462,276992073,48.306273,6664
9,JAWA TENGAH,4060.165107,11111.518048,17831.977273,100114044,3.022727,1496
